In [1]:
DEBUG = True
N_DEBUG = 100

In [2]:

# import sklearn
# import sklearn.ensemble
# import sklearn.metrics
# import sklearn.feature_extraction
# from sklearn.pipeline import make_pipeline
import pointing_game_util

In [3]:
import pandas as pd

In [4]:
test = pd.read_pickle("./dataset_test.pkl")
test = test # always load the full dataset! (np.random.shuffle(tokenized_sentences)). slice the actual hybrid_documents if debugging!
documents = test["answer"]
gold_labels = test["author"] == "human_answers" # convention: 0: machine, 1: human, see detector.py

In [5]:
from gpt2outputdataset.detector_radford import DetectorRadford
from detectgpt.detector_detectgpt import DetectorDetectGPT
from detector_guo import DetectorGuo
detector_classes = [DetectorGuo,DetectorRadford,DetectorDetectGPT]

from explainer_wrappers import LIME_Explainer, SHAP_Explainer
explainer_classes = [LIME_Explainer, ]

In [6]:
# keep a copy of the hybrid documents for the assert in the loop
ref_assert_hybrid_documents, _, _ = pointing_game_util.hybrid(documents.to_list(), gold_labels.to_list(), word_tokenizer=LIME_Explainer(DetectorRadford()).tokenize)


In [7]:
results = []
for detector_class in detector_classes:
    detector = detector_class()
    print("Initialized " + detector.__class__.__name__)
    for explainer_class in explainer_classes:
        explainer = explainer_class(detector)
        print("Initialized " + explainer.__class__.__name__)

        print("Indexing hybrid documents for " + explainer.__class__.__name__)
        hybrid_documents, tokenized_hybrid_documents, GT = pointing_game_util.hybrid(documents.to_list(), gold_labels.to_list(), word_tokenizer=explainer.tokenize)
        assert (all([a==b for a,b in zip(ref_assert_hybrid_documents,hybrid_documents)])), "(full) Hybrid documents don't match" # tokenized_hybrid_documents differ by design to make the calculation of the pointing game accuracy easier

        if DEBUG:
            hybrid_documents = hybrid_documents[0:N_DEBUG]
            tokenized_hybrid_documents = tokenized_hybrid_documents[0:N_DEBUG]
            GT = GT[0:N_DEBUG]

        print("Predicting hybrid documents")
        predictions_hybrid = detector.predict_label(hybrid_documents)

        print("Generating explanations on hybrid documents and calculating pointing game accuracy")

        
        pointing_game_acc = pointing_game_util.get_pointing_game_acc(hybrid_documents, explainer, predictions_hybrid, GT)

        print("Pointing game accuracy for {} | {}: {}".format(explainer.__class__.__name__, detector.__class__.__name__, pointing_game_acc))
        results.append((explainer.__class__.__name__, detector.__class__.__name__, pointing_game_acc))

        




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Initialized DetectorGuo
Initialized LIME_Explainer
Indexing hybrid documents for LIME_Explainer
Predicting hybrid documents
Generating explanations on hybrid documents and calculating pointing game accuracy
Pointing game accuracy for LIME_Explainer | DetectorGuo: 1.0
Initialized SHAP_Explainer
Indexing hybrid documents for SHAP_Explainer
Predicting hybrid documents
Generating explanations on hybrid documents and calculating pointing game accuracy
Pointing game accuracy for SHAP_Explainer | DetectorGuo: 0.73
Initialized DetectorRadford
Initialized LIME_Explainer
Indexing hybrid documents for LIME_Explainer
Predicting hybrid documents
Generating explanations on hybrid documents and calculating pointing game accuracy
Pointing game accuracy for LIME_Explainer | DetectorRadford: 0.53
Initialized SHAP_Explainer
Indexing hybrid documents for SHAP_Explainer
Predicting hybrid documents
Generating explanations on hybrid documents and calculating pointing game accuracy
Pointing game accuracy for 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading mask filling model t5-small...
DONE (0.06s)
Initialized
Initialized DetectorDetectGPT
Initialized LIME_Explainer
Indexing hybrid documents for LIME_Explainer
Predicting hybrid documents
DONE (0.10s)
DONE (0.09s)
Generating explanations on hybrid documents and calculating pointing game accuracy
Pointing game accuracy for LIME_Explainer | DetectorDetectGPT: 0.61
Initialized SHAP_Explainer
Indexing hybrid documents for SHAP_Explainer
Predicting hybrid documents
DONE (0.11s)
DONE (0.10s)
Generating explanations on hybrid documents and calculating pointing game accuracy
regen On the other hand, when we try to divide zero by zero, we are asking "how many times does zero go into zero?" So anything that is pictured from that scale in simulations or scanning tunneling microscopy is inherently an interpretation of data for the benefit of humans . These tissues and structures include blood vessels, muscles, and nerves that are located in and around the brain, as well as the tissues lining

  0%|          | 0/498 [00:00<?, ?it/s]

...

DONE (0.13s)
DONE (0.11s)
DONE (0.12s)
DONE (0.11s)
DONE (0.11s)
DONE (0.11s)
DONE (0.13s)
DONE (0.11s)
DONE (0.11s)
DONE (0.12s)
DONE (0.11s)
DONE (0.11s)
DONE (0.11s)
DONE (0.11s)
DONE (0.12s)
DONE (0.10s)
DONE (0.12s)
DONE (0.11s)
DONE (0.12s)
DONE (0.12s)
DONE (0.12s)
DONE (0.10s)
DONE (0.12s)
DONE (0.11s)
DONE (0.12s)
DONE (0.11s)
DONE (0.11s)
DONE (0.10s)
DONE (0.12s)
DONE (0.12s)
DONE (0.11s)
DONE (0.12s)
DONE (0.12s)
DONE (0.11s)
DONE (0.12s)
DONE (0.10s)
DONE (0.11s)
DONE (0.10s)
DONE (0.11s)
DONE (0.11s)
DONE (0.12s)
DONE (0.10s)
DONE (0.11s)
DONE (0.14s)
DONE (0.11s)
DONE (0.12s)
DONE (0.13s)
DONE (0.11s)
DONE (0.12s)
DONE (0.12s)
DONE (0.11s)
DONE (0.12s)
DONE (0.12s)
DONE (0.13s)
DONE (0.15s)
DONE (0.15s)
DONE (0.19s)
DONE (0.12s)
DONE (0.13s)
DONE (0.12s)
DONE (0.12s)
DONE (0.12s)
DONE (0.12s)
DONE (0.12s)
DONE (0.11s)
DONE (0.10s)
DONE (0.13s)
DONE (0.15s)
DONE (0.13s)
DONE (0.12s)
DONE (0.13s)
DONE (0.12s)
DONE (0.12s)
DONE (0.11s)
DONE (0.31s)
DONE (0.13s)
DONE (0.13s)

PartitionExplainer explainer: 2it [05:18, 318.64s/it]              


regen You can contribute it to the corporation, but it is another tax event and you should understand fully the consequences. However, it's important to remember that drinking alcohol can also have negative consequences. At a certain temperature, called the freezing point, the molecules will line up in a specific pattern, forming a solid.\n\nWhen water is a liquid, the molecules are moving around and are able to flow freely. Forensic scientists can sometimes tie a bullet to a specific firearm by examining the markings that the gun leaves on the bullet as it is fired. Diamonds are a common gemstone , and easily grown in a lab . Additionally, Native American cultures were impacted by European colonization, which disrupted their way of life and made it difficult for them to maintain their traditional way of life. I realise that does n't quite answer your question , but it sorta makes sense to me . > I guess what I am asking is how was the first diamond cutter cut into a diamond cutter Dia

  0%|          | 0/498 [00:00<?, ?it/s]

DONE (0.15s)
DONE (0.11s)
DONE (0.11s)
DONE (0.20s)
DONE (0.14s)
DONE (0.12s)
DONE (0.14s)
DONE (0.18s)
DONE (0.13s)
DONE (0.19s)
DONE (0.26s)
DONE (0.14s)
DONE (0.17s)
DONE (0.12s)
DONE (0.13s)


In [ ]:
df = pd.DataFrame(results, columns=["Explanation Method", "Detector", "Pointing Game ACC"])
df

In [ ]:
# import shap

In [ ]:

# for hybrid_document in hybrid_documents:

#     explainer = LIME_Explainer(detector)
#     explainer.get_explanation_cached(hybrid_document).show_in_notebook()

#     explainer = SHAP_Explainer(detector)
#     shap.text_plot(explainer.get_explanation_cached(hybrid_document))
        